In [7]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [6]:
A = 0.86
B = 0.16
C = 1.
dt = 1e-4
u_t = 0. 

In [9]:
def system(x_t, u_t, A, B, C):
    x_t = A * x_t + B * u_t
    return C * x_t

def error(Z_t, Z):
    return abs(Z_t - Z)

def signal_z_ref(N):
    Z_ref = []
    for i in range(N):
        if i < N / 2:
            Z_ref.append(0)
        else:
            Z_ref.append(1)
    return Z_ref



In [8]:
# Define class to add andom noise for data (Ornstein–Uhlenbeck process)

class OUActionNoise:
    def __init__(self, mean, std_deviation, theta=0.0015, dt=1e-4, x_initial=None):
        self.theta = theta
        self.mean = mean
        self.std_dev = std_deviation
        self.dt = dt
        self.x_init = x_initial
        self.reset()
    
    def __call__(self):
        # Formula is taken from https://en.wikipedia.org/wiki/Ornstein%E2%80%93Uhlenbeck_process
        x = (
            self.x_prev + self.theta * (self.mean - self.x_prev) * self.dt
            + self.std_dev + np.sqrt(self.dt) * np.random.normal(size=self.mean.shape)
        )
        self.x_prev = x
        return x

    def reset(self):
        if self.x_init is not None:
            self.x_prev = self.x_init
        else:
            self.x_prev = np.zeros_like(self.mean)

In [10]:
std_dev = 1e-3
ou_noise = OUActionNoise(mean=np.zeros(1), std_deviation=float(std_dev) * np.ones(1))

In [ ]:
def define_x_t(N):
    x_t = []
    for i in range(N):
        noise = float(ou_noise() * 1e-2)
        x_t.append(noise)
    noise = np.array(x_t)